# 迁移指南 - 从 v0.21 到 v1.0.0

Gymnasium 是OpenAI Gym v0.26 的分支，它引入了与 Gym v0.21 相比的重大变化。在本指南中，我们简要概述了从 Gym v0.21（许多教程都是为这个版本编写的）到 Gym v0.26 的 API 变化。对于仍然停留在 v0.21 API 中的环境，请参阅本指南。

:::::{grid} 2
::::{grid-item-card}  v0.21 版本
```python
import gym
env = gym.make("LunarLander-v3", options={})
env.seed(123)
observation = env.reset()

done = False
while not done:
    action = env.action_space.sample()  # agent policy that uses the observation and info
    observation, reward, done, info = env.step(action)

    env.render(mode="human")

env.close()
```
::::
::::{grid-item-card}  v0.26
```python
import gym
env = gym.make("LunarLander-v3", render_mode="human")
observation, info = env.reset(seed=123, options={})

done = False
while not done:
    action = env.action_space.sample()  # agent policy that uses the observation and info
    observation, reward, terminated, truncated, info = env.step(action)

    done = terminated or truncated

env.close()
```
::::
:::::

## 种子和随机数生成器

Gym v0.26 环境中的 `Env.seed()` 已被移除，改为使用 `Env.reset(seed=seed)`。这允许仅在环境重置时更改种子。移除种子的决定是因为一些环境使用模拟器，这些模拟器无法在一个回合内更改随机数生成器，必须在新回合开始时进行。我们知道有些情况下控制随机数生成器很重要，在这些情况下，如果环境使用内置的随机数生成器，用户可以使用 {attr}`~gymnasium.Env.np_random` 属性手动设置种子。

Gymnasium v0.26 改为使用 {class}`numpy.random.Generator` 而不是自定义随机数生成器。这意味着一些函数如randint被移除，转而使用整数。虽然一些环境可能使用外部随机数生成器，我们建议使用包装器和外部用户可以访问和使用的 {attr}`~gymnasium.Env.np_random` 属性。

## 环境重置

在 v0.26 中，{func}`~gymnasium.reset` 接受两个可选参数并返回一个值。这与 v0.21 不同，后者不接受参数且返回 `None`。这两个参数分别是用于设置随机数生成器的 `seed` 和允许在重置时向环境传递额外数据的 `options`。例如，在经典控制中，`options` 参数现在允许用户修改状态界限的范围。更多细节请参见[原始 PR](https://github.com/openai/gym/pull/2921)。

{func}`~gymnasium.Env.reset` 进一步返回 `info`，类似于 {func}`~gymnasium.Env.step` 返回的 `info`。这一点很重要，因为 `info` 可以包含在下一步中使用或保存的指标或有效动作掩码。

为了更新较旧的环境，我们强烈建议在 {func}`~gymnasium.Env.reset` 的第一行调用 `super().reset(seed=seed)`。这将自动使用种子值更新 {attr}`~gymnasium.Env.np_random`。

## 环境步进

在 v0.21 中，{func}`~gymnasium.Env.step` 的类型定义为 `tuple[ObsType, SupportsFloat, bool, dict[str, Any]]`，表示下一步观察、步骤奖励、回合是否结束以及步骤的额外信息。由于可再现性问题（将在即将发布的博客文章中详细讨论），我们将类型定义更改为 `tuple[ObsType, SupportsFloat, bool, bool, dict[str, Any]]`，增加了额外的布尔值。这个额外的 `bool` 对应于旧的 `done`，现更改为 `terminated` 和 `truncated`。这些变更在 Gym v0.26 中引入（在 `v25` 中默认关闭）。

对于希望更新的用户，在大多数情况下，将step()中的 `done` 替换为 `terminated` 和 `truncated=False` 应能解决大部分问题。然而，有回合截断而非终止原因的环境应该阅读相关PR。对于循环遍历环境的用户，他们应修改 `done = terminated` 或者 `truncated`，如示例代码所示。对于训练库，主要区别在于将 `done` 更改为 `terminated`，以指示是否应发生引导。

## 时间限制包装器

在 v0.21 中，`TimeLimit` 包装器在 `info` 字典中添加了一个额外的键 `TimeLimit.truncated`，每当代理达到时间限制而未达到终端状态时。

在 v0.26 中，此信息通过前面部分描述的 `truncated` 返回值传达，如果代理达到时间限制，无论是否达到终端状态，该值为 `True`。旧字典条目等价于 `truncated` 且未 `terminated`。

## 环境渲染

在v0.26中，引入了新的渲染API，使得渲染模式在初始化时固定，因为一些环境不允许实时更改渲染模式。因此，用户现在应在 `gym.make` 中指定 {attr}`~gymnasium.Env.render_mode`，如上面的v0.26示例代码所示。

有关变更的更完整解释，请参阅此[总结](https://younis.dev/blog/render-api/)。

## 已移除的代码

- `GoalEnv` - 已被移除，需要它的用户应重新实现环境或使用包含此环境实现的 [Gymnasium Robotics](https://robotics.farama.org/)。
- `from gym.envs.classic_control import rendering` - 已被移除，转而由用户实现自己的渲染系统。Gymnasium 环境使用 {mod}`pygame` 编写。
- 机器人环境 - 机器人环境已移至 Gymnasium Robotics 项目。
- Monitor 包装器 - 此包装器被 {class}`~gymnasium.wrappers.RecordVideo` 和 {class}`~gymnasium.wrappers.RecordEpisodeStatistics` 两个独立包装器取代。